# PyTorch RNN

[RNN Doc PyTorch](https://docs.pytorch.org/docs/stable/generated/torch.nn.RNN.html#rnn)

**Parameters:**

- input_size – The number of expected features in the input x
- hidden_size – The number of features in the hidden state h

- batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) 
- instead of (seq, batch, feature). 

Note that this does not apply to hidden or cell states. 

# Generating a Synthetic Weather Time-Series

To train an RNN, we need a sequence with memory. Weather is a natural example: 
today’s temperature depends on previous days plus some randomness.

We generate a simple autoregressive process:

`T_t = 0.7 * T_(t-1) + 0.2 * T_(t-2) + Normal(0, 0.5)`

This produces a smooth temperature sequence with short-term memory and noise.
We will use the first 5 days to predict day 6.


In [1]:
import numpy as np

temps = []
temps.append(20)   # seed day 0
temps.append(21)   # seed day 1

for t in range(2, 100):
    next_temp = (
        0.7 * temps[-1] +
        0.2 * temps[-2] +
        np.random.normal(0, 0.5)
    )
    temps.append(next_temp)

input_sq = temps[:5]
target = temps[5]
print(input_sq, target)

[20, 21, 18.28143859043176, 17.268550713262034, 15.88441994345271] 14.620642581377346


In [5]:
import torch.nn as nn
import torch
import torch.optim as optim


class RNNPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=20):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, h = self.rnn(x)
        last_h = out[:, -1, :]   # final hidden state
        y_pred = self.fc(last_h)
        return y_pred


## Make the dataset tensor instead of np array

In [6]:
def make_dataset(data, seq_len=5):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

X, y = make_dataset(temps, seq_len=5)

# reshape for PyTorch: (batch, seq_len, input_size)
X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

print(X.shape, y.shape)
print(X[1].shape)

torch.Size([95, 5, 1]) torch.Size([95, 1])
torch.Size([5, 1])


### input: tensor of shape (L, H_in)
### hx tensor of shape (D * num_layers, H_out)

- L = sequence length
- H_in = input size
- D = defult **(1)** 2 if bidirectional=True
- H_out = hidden size

## Just like before 

- define the model
- define the loss_fn
- define the optimizer
- write the training loop

In [7]:
model = RNNPredictor(input_size=1, hidden_size=20)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = model(X)
    loss = loss_fn(y_pred, y)

    loss.backward()
    optimizer.step()

    if epoch % 300 == 0:
        print(f"epoch {epoch}, loss={loss.item():.4f}")


epoch 0, loss=17.7383
epoch 300, loss=0.2215
epoch 600, loss=0.1898
epoch 900, loss=0.1098
epoch 1200, loss=0.0668
epoch 1500, loss=0.0380
epoch 1800, loss=0.0176
epoch 2100, loss=0.0124
epoch 2400, loss=0.0061
epoch 2700, loss=0.0066


In [9]:
test_seq = torch.tensor(temps[:5]).unsqueeze(0).unsqueeze(-1)
pred = model(test_seq)

print("Input:", temps[:5])
print("Prediction:", pred.item())
print("True next value:", temps[5])

Input: [20, 21, 18.28143859043176, 17.268550713262034, 15.88441994345271]
Prediction: 14.602416038513184
True next value: 14.620642581377346
